In [145]:
import os
from estnltk import Text
from estnltk_neural.taggers import StanzaSyntaxTagger
import pandas as pd
import numpy as np
import sklearn
import csv

Alustuseks fraasimustrid, mis peaksid olema korrektsed. Fraasimustrid on tekstifailis, igal real üks muster. Esituskuju on järgnev: id-ner/timex/other-ülemuse_id-deprel-POS eraldatuna tühikutega. Eri sõnade info on eraldatud komadega. Juhul, kui POS on mitmene, esitatakse variandid püstkriipsuga eraldatult. Indekseid mustrites ei esitata, sest fraasisiseste otsingut korral võivad tekkida ebakõlad. Liikmete järjestus vastab järjestusele tekstis.



Siin võetakse aga kasutusele variant, kus ID-d puuduvad, sest fraaside seest mustreid otsides võib tekkida ID-dega ebakõlasid.

In [126]:
# patterns are kept in a list of strings
patterns = None

with open('correct_tree_patterns_no_IDs.txt', 'r') as f:
    patterns = [line.rstrip() for line in f]

for pattern in patterns:
    print(pattern)

PER root H,OTHER flat H,OTHER flat H
LOC nmod G,OTHER root S
LOC nmod H,OTHER root D
LOC nmod H,OTHER root H
LOC nmod H,OTHER root S
ORG nmod S,OTHER root S
ORG nmod Y,OTHER root S
LOC amod G,OTHER root S
LOC amod H,OTHER root S
LOC nmod H,OTHER root S,OTHER appos H,OTHER flat H
ORG nmod H,OTHER root S,OTHER appos H,OTHER flat S
LOC nmod H,OTHER root S,OTHER appos H,OTHER flat Y
LOC nmod H,OTHER root S,OTHER appos S,OTHER flat H
ORG nmod S,OTHER root S,OTHER appos H,OTHER flat H
ORG nmod S,OTHER root S,OTHER appos S,OTHER flat H
LOC nmod Y,OTHER root S,OTHER appos H,OTHER flat H
LOC obl H,OTHER acl A,OTHER root S
PER nmod H,OTHER root S,OTHER nmod S
LOC obl A,OTHER acl A,OTHER root S
LOC obl Y,OTHER acl A,OTHER root S
OTHER advmod D|I,PER nmod H,OTHER root S
OTHER advmod D,LOC nmod H,OTHER root S
OTHER advmod S,LOC nmod H,OTHER root S
LOC amod H,OTHER amod A,OTHER root S
LOC amod H,OTHER amod O,OTHER root S
OTHER amod A,LOC nmod G,OTHER root S
OTHER amod A,LOC nmod H,OTHER root S
TIMEX

In [77]:
df = pd.read_pickle('phrase_examples_filtered_sub10000/atomic_phrases_reverse_top50_ner_timex.pkl')

In [78]:
df

,phrase,phrase_length,document_creation_time,sentence_id,document_id,sentence_startend,subcorpus,phrase_type,phrase_start_end,has_ner_netity,...,pos_sequence,graph,graph_code,graph_code_pos,graph_code_pos_ner_timex,pos_sequence_verb_info,graph_verb_info,graph_code_verb_info,graph_code_pos_verb_info,graph_code_pos_ner_timex_verb_info
0,Text(text='kütusefirma Olerex turundusjuhina'),3,2024-03-10T00:49,13840078,610603,"(1428, 1569)",aja_sloleht,obl_phrase,"(106, 139)",1,...,S-H-S,"(1, 0, 2, 3)","((0, 1, root),(1, 2, appos),(2, 3, flat))","(S-H-S,(0, 1, root),(1, 2, appos),(2, 3, flat))","(0-ner_ORG-0,S-H-S,(0, 1, root),(1, 2, appos),(2, 3, flat))",S-H-S,"(1, 0, 2, 3)","((0, 1, root),(1, 2, appos),(2, 3, flat))","(S-H-S,(0, 1, root),(1, 2, appos),(2, 3, flat))","(0-ner_ORG-0,S-H-S,(0, 1, root),(1, 2, appos),(2, 3, flat))"
1,Text(text='Rootsi Toomas Tiiveli'),3,2024-03-10T01:05,1240867,46774,"(6468, 6667)",aja_EPL,obj_phrase,"(30, 51)",1,...,H-H-H,"(1, 0, 2, 3)","((0, 1, root),(1, 2, appos),(2, 3, flat))","(H-H-H,(0, 1, root),(1, 2, appos),(2, 3, flat))","(ner_LOC-0-0,H-H-H,(0, 1, root),(1, 2, appos),(2, 3, flat))",H-H-H,"(1, 0, 2, 3)","((0, 1, root),(1, 2, appos),(2, 3, flat))","(H-H-H,(0, 1, root),(1, 2, appos),(2, 3, flat))","(ner_LOC-0-0,H-H-H,(0, 1, root),(1, 2, appos),(2, 3, flat))"
2,Text(text='Kevadel HC Kehra'),3,2024-03-10T01:18,3601084,186664,"(261, 391)",aja_EPL,obl_phrase,"(0, 16)",0,...,S-Y-H,"(1, 0, 2, 3)","((0, 1, root),(1, 2, appos),(2, 3, flat))","(S-Y-H,(0, 1, root),(1, 2, appos),(2, 3, flat))","(timex-0-0,S-Y-H,(0, 1, root),(1, 2, appos),(2, 3, flat))",S-Y-H,"(1, 0, 2, 3)","((0, 1, root),(1, 2, appos),(2, 3, flat))","(S-Y-H,(0, 1, root),(1, 2, appos),(2, 3, flat))","(timex-0-0,S-Y-H,(0, 1, root),(1, 2, appos),(2, 3, flat))"
3,Text(text='Vilniuses John A. Cloud'),3,2024-03-10T01:29,14042955,618671,"(0, 172)",aja_sloleht,appos_phrase,"(147, 170)",1,...,H-H-H,"(1, 0, 2, 3)","((0, 1, root),(1, 2, appos),(2, 3, flat))","(H-H-H,(0, 1, root),(1, 2, appos),(2, 3, flat))","(ner_LOC-0-0,H-H-H,(0, 1, root),(1, 2, appos),(2, 3, flat))",H-H-H,"(1, 0, 2, 3)","((0, 1, root),(1, 2, appos),(2, 3, flat))","(H-H-H,(0, 1, root),(1, 2, appos),(2, 3, flat))","(ner_LOC-0-0,H-H-H,(0, 1, root),(1, 2, appos),(2, 3, flat))"
4,Text(text='Kristina vahele'),2,2024-03-10T00:45,14409648,630860,"(883, 1088)",aja_sloleht,obl_phrase,"(41, 56)",1,...,H-K,"(1, 0, 2)","((0, 1, root),(1, 2, case))","(H-K,(0, 1, root),(1, 2, case))","(ner_PER-0,H-K,(0, 1, root),(1, 2, case))",H-K,"(1, 0, 2)","((0, 1, root),(1, 2, case))","(H-K,(0, 1, root),(1, 2, case))","(ner_PER-0,H-K,(0, 1, root),(1, 2, case))"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
900,Text(text='mu Tartu korterisse'),3,2024-03-10T01:24,18883609,704369,"(7977, 8120)",aja_ml,obl_phrase,"(34, 53)",1,...,P-H-S,"(1, 3, 2, 0)","((0, 3, root),(3, 1, nmod),(3, 2, nmod))","(S-P-H,(0, 3, root),(3, 1, nmod),(3, 2, nmod))","(0-0-ner_LOC,S-P-H,(0, 3, root),(3, 1, nmod),(3, 2, nmod))",P-H-S,"(1, 3, 2, 0)","((0, 3, root),(3, 1, nmod),(3, 2, nmod))","(S-P-H,(0, 3, root),(3, 1, nmod),(3, 2, nmod))","(0-0-ner_LOC,S-P-H,(0, 3, root),(3, 1, nmod),(3, 2, nmod))"
901,Text(text='oma Tartu kodus'),3,2024-03-10T01:29,9228266,422383,"(220, 446)",aja_pm,obl_phrase,"(20, 35)",1,...,P-H-S,"(1, 3, 2, 0)","((0, 3, root),(3, 1, nmod),(3, 2, nmod))","(S-P-H,(0, 3, root),(3, 1, nmod),(3, 2, nmod))","(0-0-ner_LOC,S-P-H,(0, 3, root),(3, 1, nmod),(3, 2, nmod))",P-H-S,"(1, 3, 2, 0)","((0, 3, root),(3, 1, nmod),(3, 2, nmod))","(S-P-H,(0, 3, root),(3, 1, nmod),(3, 2, nmod))","(0-0-ner_LOC,S-P-H,(0, 3, root),(3, 1, nmod),(3, 2, nmod))"
902,Text(text='/ BNS kuninganna'),3,2024-03-10T01:09,12955414,571811,"(2136, 2331)",aja_sloleht,nmod_phrase,"(164, 179)",1,...,Z-Y-S,"(1, 3, 2, 0)","((0, 3, root),(3, 1, punct),(3, 2, nmod))","(S-Z-Y,(0, 3, root),(3, 1, punct),(3, 2, nmod))","(0-0-ner_ORG,S-Z-Y,(0, 3, root),(3, 1, punct),(3, 2, nmod))",Z-Y-S,"(1, 3, 2, 0)","((0, 3, root),(3, 1, punct),(3, 2

In [81]:
test = None
for idx, row in df.iterrows():
    test = row['phrase']
    break

In [82]:
test.stanza_syntax

Layer(name='stanza_syntax', attributes=('id', 'lemma', 'upostag', 'xpostag', 'feats', 'head', 'deprel', 'deps', 'misc', 'parent_span', 'children'), spans=SL[Span('kütusefirma', [{'id': 1, 'lemma': 'kütusefirma', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('n', 'n')]), 'head': 0, 'deprel': 'root', 'deps': '_', 'misc': '_', 'parent_span': None, 'children': <class 'tuple'>}]),
Span('Olerex', [{'id': 2, 'lemma': 'Olerex', 'upostag': 'H', 'xpostag': 'H', 'feats': OrderedDict([('sg', 'sg'), ('n', 'n')]), 'head': 1, 'deprel': 'appos', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]),
Span('turundusjuhina', [{'id': 3, 'lemma': 'turundusjuht', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('es', 'es')]), 'head': 2, 'deprel': 'flat', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}])])

In [21]:
def get_ner_timex(text_obj, stanza_word):
    ner = None
    nertag = None
    if len(text_obj.ner) > 0:
        word = text_obj.words.get(stanza_word)
        for n in text_obj.ner:
            for part in n:
                if part==word:
                    ner=word
                    nertag=n.nertag
    timex = text_obj.timexes.get(stanza_word)
    if ner:
        return nertag
    if timex:
        return 'TIMEX'
    return 'OTHER'

In [24]:
def get_unique_POS(word):
    #pos_list = word.morph_analysis['partofspeech']
    infinite_verb_forms = ['da', 'des', 'ma', 'maks', 'mas', 'mast', 'mata', 'nud', 'tav', 'tud', 'v']
    # if POS is ambiguous, only unique tags are kept, e.g. ['V', 'A', 'A'] -> ['V', 'A']
    pos_list = []
    for i in range(len(word.morph_analysis['partofspeech'])):
        if word.morph_analysis['partofspeech'][i] == 'V':
            if word.morph_analysis['form'][i] in infinite_verb_forms:
                pos_list.append('V_inf')
            elif word.form[i] == 'neg':
                pos_list.append('V_neg')
            else:
                pos_list.append('V_fin')
        else:
            pos_list.append(word.morph_analysis['partofspeech'][i])
    
    if len(pos_list) > 1:
        char_unique = [char for indx, char in enumerate(pos_list) if char not in pos_list[:indx]]
        if len(char_unique) < 2:
            return char_unique[0]
        return '|'.join(char_unique)
    return pos_list[0]

Naiivne lähenemine: vaatab vaid variante, kus liikmed paiknevad järjest:

In [83]:
pattern_matches_found = []

for i in range(len(test.stanza_syntax)-1):
    pattern_found = []
    ner_timex = get_ner_timex(test, test.stanza_syntax[i])
    pos = get_unique_POS(test.stanza_syntax[i])
    pattern_found.append(f'{test.stanza_syntax[i].id} {ner_timex} {test.stanza_syntax[i].head} {test.stanza_syntax[i].deprel} {pos}')
    for j in range(i+1, len(test.stanza_syntax)):
        ner_timex = get_ner_timex(test, test.stanza_syntax[j])
        pos = get_unique_POS(test.stanza_syntax[j])
        pattern_found.append(f'{test.stanza_syntax[j].id} {ner_timex} {test.stanza_syntax[j].head} {test.stanza_syntax[j].deprel} {pos}')
        if ','.join(pattern_found) in patterns:
            pattern_matches_found += ','.join(pattern_found)
        if j == len(test.stanza_syntax):
            pattern_found = []
            break
        

In [84]:
print(','.join(pattern_found))

2 ORG 1 appos H,3 OTHER 2 flat S


In [107]:
test2 = None
for idx, row in df.iterrows():
    if idx == 1:
        test2 = row['phrase']
        break

In [108]:
test2.stanza_syntax

Layer(name='stanza_syntax', attributes=('id', 'lemma', 'upostag', 'xpostag', 'feats', 'head', 'deprel', 'deps', 'misc', 'parent_span', 'children'), spans=SL[Span('Rootsi', [{'id': 1, 'lemma': 'Rootsi', 'upostag': 'H', 'xpostag': 'H', 'feats': OrderedDict([('sg', 'sg'), ('n', 'n')]), 'head': 0, 'deprel': 'root', 'deps': '_', 'misc': '_', 'parent_span': None, 'children': <class 'tuple'>}]),
Span('Toomas', [{'id': 2, 'lemma': 'Toomas', 'upostag': 'H', 'xpostag': 'H', 'feats': OrderedDict([('sg', 'sg'), ('n', 'n')]), 'head': 1, 'deprel': 'appos', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]),
Span('Tiiveli', [{'id': 3, 'lemma': 'Tiiveli', 'upostag': 'H', 'xpostag': 'H', 'feats': OrderedDict([('sg', 'sg'), ('n', 'n')]), 'head': 2, 'deprel': 'flat', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}])])

In [103]:
pattern_matches_found = []

for i in range(len(test.stanza_syntax)-1):
    pattern_members = []
    pattern_found = []

    if i == 0:
        pattern_members.append(test.stanza_syntax[i])
        
    else:
        for member in pattern_members:
            
            if test.stanza_syntax[i] in member.children or member in test.stanza_syntax[i].children:
                pattern_members.append(test.stanza_syntax[i])
                ner_timex = get_ner_timex(test, test.stanza_syntax[i])
                pos = get_unique_POS(test.stanza_syntax[i])
                pattern_found.append(f'{test.stanza_syntax[i].id} {ner_timex} {test.stanza_syntax[i].head} {test.stanza_syntax[i].deprel} {pos}')
                
                if ','.join(pattern_found) in patterns:
                    pattern_matches_found.append(','.join(pattern_found))
                    
    for j in range(i+1, len(test.stanza_syntax)):
        
        for member in pattern_members:
            
            if test.stanza_syntax[j] in member.children or member in test.stanza_syntax[j].children:
                pattern_members.append(test.stanza_syntax[j])
                ner_timex = get_ner_timex(test, test.stanza_syntax[j])
                pos = get_unique_POS(test.stanza_syntax[j])
                pattern_found.append(f'{test.stanza_syntax[j].id} {ner_timex} {test.stanza_syntax[j].head} {test.stanza_syntax[j].deprel} {pos}')
            
            if ','.join(pattern_found) in patterns:
                pattern_matches_found.append(','.join(pattern_found))
                
            if j == len(test.stanza_syntax) - 1:
                pattern_members = []
                pattern_found = []
                break
                
        else:
            continue

In [104]:
print(pattern_matches_found)

[]


In [130]:
pattern_matches_found = []

for i in range(len(test2.stanza_syntax) - 1):
    pattern_members = []
    pattern_found = []

    pattern_members.append(test2.stanza_syntax[i])
    ner_timex = get_ner_timex(test2, test2.stanza_syntax[i])
    pos = get_unique_POS(test2.stanza_syntax[i])
    pattern_found.append(f"{ner_timex} {test2.stanza_syntax[i].deprel} {pos}")

    for j in range(i + 1, len(test2.stanza_syntax)):
        for member in pattern_members:
            print(member)
            print(test2.stanza_syntax[j])
            print(test2.stanza_syntax[j] in member.children)
            if test2.stanza_syntax[j] in member.children or member in test2.stanza_syntax[j].children:
                pattern_members.append(test2.stanza_syntax[j])
                ner_timex = get_ner_timex(test2, test2.stanza_syntax[j])
                pos = get_unique_POS(test2.stanza_syntax[j])
                pattern_found.append(f"{ner_timex} {test2.stanza_syntax[j].deprel} {pos}")
                
                print(",".join(pattern_found))
                print(",".join(pattern_found) in patterns)
                if ",".join(pattern_found) in patterns:
                    pattern_matches_found.append(",".join(pattern_found))

                if j == len(test2.stanza_syntax) - 1:
                    pattern_members = []
                    pattern_found = []
                    break


Span('Rootsi', [{'id': 1, 'lemma': 'Rootsi', 'upostag': 'H', 'xpostag': 'H', 'feats': OrderedDict([('sg', 'sg'), ('n', 'n')]), 'head': 0, 'deprel': 'root', 'deps': '_', 'misc': '_', 'parent_span': None, 'children': <class 'tuple'>}])
Span('Toomas', [{'id': 2, 'lemma': 'Toomas', 'upostag': 'H', 'xpostag': 'H', 'feats': OrderedDict([('sg', 'sg'), ('n', 'n')]), 'head': 1, 'deprel': 'appos', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}])
True
LOC root H,OTHER appos H
False
Span('Toomas', [{'id': 2, 'lemma': 'Toomas', 'upostag': 'H', 'xpostag': 'H', 'feats': OrderedDict([('sg', 'sg'), ('n', 'n')]), 'head': 1, 'deprel': 'appos', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}])
Span('Toomas', [{'id': 2, 'lemma': 'Toomas', 'upostag': 'H', 'xpostag': 'H', 'feats': OrderedDict([('sg', 'sg'), ('n', 'n')]), 'head': 1, 'deprel': 'appos', 'deps': '_', 'misc': '_', 'parent_s

In [131]:
print(pattern_matches_found)

[]


### On more data

In [142]:
def find_patterns(phrase):
    pattern_matches_found = []

    for i in range(len(phrase.stanza_syntax) - 1):
        
        pattern_members = []
        pattern_found = []

        pattern_members.append(phrase.stanza_syntax[i])
        ner_timex = get_ner_timex(phrase, phrase.stanza_syntax[i])
        pos = get_unique_POS(phrase.stanza_syntax[i])
        pattern_found.append(f"{ner_timex} {phrase.stanza_syntax[i].deprel} {pos}")

        for j in range(i + 1, len(phrase.stanza_syntax)):
            
            for member in pattern_members:
                
                if phrase.stanza_syntax[j] in member.children or member in phrase.stanza_syntax[j].children:
                    pattern_members.append(phrase.stanza_syntax[j])
                    ner_timex = get_ner_timex(phrase, phrase.stanza_syntax[j])
                    pos = get_unique_POS(phrase.stanza_syntax[j])
                    pattern_found.append(f"{ner_timex} {phrase.stanza_syntax[j].deprel} {pos}")
                
                    if ",".join(pattern_found) in patterns:
                        to_be_added = []
                        to_be_added.append(",".join(pattern_found))
                        to_be_added.append(" ".join([member.text for member in pattern_members]))
                        pattern_matches_found.append(to_be_added)
                        #pattern_matches_found.append(",".join(pattern_found))
                        #pattern_matches_found[",".join(pattern_found)].append(" ".join([member.text for member in pattern_members]))

                    if j == len(phrase.stanza_syntax) - 1:
                        pattern_members = []
                        pattern_found = []
                        break
    
    return pattern_matches_found

In [143]:
subset = df[650:800]

for idx, row in subset.iterrows():
    print(find_patterns(row['phrase']))

[['LOC nmod H,OTHER root S', 'Läti tütarfirma'], ['LOC nmod H,OTHER root S,OTHER appos S,OTHER flat H', 'Läti tütarfirma Pluss Punkts']]
[['ORG nmod H,OTHER root S,OTHER appos H,OTHER flat S', 'Riigikogu liige Mart Nutt']]
[['LOC nmod H,OTHER root S', 'Pärnu abilinnapea'], ['LOC nmod H,OTHER root S,OTHER appos H,OTHER flat H', 'Pärnu abilinnapea Margus Tammekivi']]
[['LOC nmod H,OTHER root S', 'Soome firmaga'], ['LOC nmod H,OTHER root S,OTHER appos H,OTHER flat Y', 'Soome firmaga K . Jousmaa KY']]
[['LOC nmod Y,OTHER root S,OTHER appos H,OTHER flat H', 'USA presidendile George W. Bushile']]
[['LOC nmod H,OTHER root S', 'Briti kuninganna'], ['LOC nmod H,OTHER root S,OTHER appos H,OTHER flat Y', 'Briti kuninganna Elizabeth I']]
[['LOC nmod H,OTHER root S', 'Viljandi kriminaalpolitseiniku'], ['LOC nmod H,OTHER root S,OTHER appos H,OTHER flat H', 'Viljandi kriminaalpolitseiniku Ahti Oleski']]
[['LOC nmod H,OTHER root S', 'Portugali välisminister'], ['LOC nmod H,OTHER root S,OTHER appos H,O

In [149]:
with open('correct_pattern_matches.csv', 'w', newline='', encoding='UTF-8') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=['pattern', 'phrase'])
    writer.writeheader()
    header2 = {'pattern': 'string', 'phrase': 'string'}
    writer.writerow(header2)
    for idx, row in df.iterrows():
        matches = find_patterns(row['phrase'])
        if len(matches) > 0:
            for match in matches:
                writer.writerow({'pattern': match[0], 'phrase': match[1]})